In [1]:
from abc import ABC, abstractmethod

class Experiment(ABC):
    """
    抽象基类，定义实验的公共接口。
    """

    def __init__(self, name: str, config: dict):
        self.name = name
        self.config = config

    @abstractmethod
    def run(self) -> dict:
        """
        核心抽象方法，子类必须实现：
        - 根据 self.config 执行步骤
        - 返回实验结果（字典形式）
        """
        pass

    @classmethod
    def from_dict(cls, info: dict) -> 'Experiment':
        """
        工厂方法：根据配置字典 info 创建正确的子类实例。
        info 结构示例：
        {
            "type": "chemistry",  # 或 "physics"
            "name": "酸碱滴定",
            "config": { ... }
        }
        """
        exp_type = info.get("type")
        # 把不同类型映射到不同子类
        mapping = {
            "chemistry": ChemistryExperiment,
            "physics": PhysicsExperiment,
        }
        SubClass = mapping.get(exp_type)
        if SubClass is None:
            raise ValueError(f"未知实验类型：{exp_type}")
        return SubClass(info["name"], info["config"])

    @staticmethod
    def validate_parameters(params: dict) -> bool:
        """
        工具函数：校验传入的 config 参数是否全为正数
        （与实例 or 类状态都无关）。
        """
        return all((isinstance(v, (int, float)) and v > 0) for v in params.values())
    
class ChemistryExperiment(Experiment):
    def run(self) -> dict:
        # 先校验
        if not Experiment.validate_parameters(self.config):
            raise ValueError("化学实验参数必须为正数")
        # 假设 config 包含浓度、体积等
        conc = self.config["concentration"]  # mol/L
        vol_ml = self.config["volume_ml"]    # mL

        # 单位转换：mL -> L
        vol_l = ChemistryExperiment.ml_to_l(vol_ml)

        # 简单模拟：n = C * V
        moles = conc * vol_l
        return {
            "moles_produced": moles,
            "details": f"{conc} mol/L × {vol_l} L = {moles} mol"
        }

    @staticmethod
    def ml_to_l(ml: float) -> float:
        """静态方法：毫升到升的转换"""
        return ml / 1000.0


class PhysicsExperiment(Experiment):
    def run(self) -> dict:
        # 先校验
        if not Experiment.validate_parameters(self.config):
            raise ValueError("物理实验参数必须为正数")
        # 假设 config 包含质量 m (kg) 和加速度 a (m/s²)
        m = self.config["mass"]
        a = self.config["acceleration"]
        # F = m * a
        force = m * a
        return {
            "force": force,
            "details": f"F = {m} kg × {a} m/s² = {force} N"
        }


In [2]:
def main():
    # 1. 准备实验配置（通常从 JSON 或数据库读取）
    exp_info = {
        "type": "chemistry",
        "name": "滴定实验",
        "config": {
            "concentration": 0.1,   # mol/L
            "volume_ml": 50         # mL
        }
    }

    # 2. 用 classmethod 工厂方法创建实例
    exp = Experiment.from_dict(exp_info)
    print(f"已创建实验：{exp.name}，类型：{type(exp).__name__}")

    # 3. 运行实验
    result = exp.run()
    print("实验结果：", result)

if __name__ == "__main__":
    main()


已创建实验：滴定实验，类型：ChemistryExperiment
实验结果： {'moles_produced': 0.005000000000000001, 'details': '0.1 mol/L × 0.05 L = 0.005000000000000001 mol'}
